In [256]:
import pandas as pd
import numpy as np
from sklearn import model_selection
import warnings

warnings.filterwarnings("ignore")

In [316]:
df = pd.read_csv('clean_data.csv')
df.head()

,Адрес,Жилая,Кухня,Общая,Цена,Цена за квадрат,Этаж,Этажность здания,Время,Способ передвижения,Широта,Долгота
0,"Московская область, Мытищи, мкр. Ярославский, ...",24.0,10.0,48.0,6550000.0,136458.0,12,25,15.0,на транспорте,55.913162,37.713063
1,"Москва, ЗАО, р-н Дорогомилово, Кутузовский про...",45.0,NaN,64.0,26000000.0,406250.0,4,7,13.0,пешком,55.750727,37.562272
2,"Москва, ЦАО, р-н Тверской, 1-й Колобовский пер...",23.4,8.9,40.0,18000000.0,450000.0,2,3,5.0,пешком,55.769216,37.617411
3,"Москва, ЗАО, р-н Раменки, Мосфильмовская ул., 8",63.0,20.0,98.0,59900000.0,611224.0,33,54,11.0,на транспорте,55.723257,37.527660
4,"Москва, ЦАО, р-н Хамовники, Комсомольский прос...",48.0,NaN,67.3,37900000.0,563150.0,4,5,10.0,пешком,55.721731,37.572899


In [317]:
df['Цена'] = df['Цена'] / 1_000_000

In [318]:
df= df.drop(columns='Адрес', axis=1)

In [319]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df['Способ передвижения'] = le.fit_transform(df['Способ передвижения'])

In [320]:
df.head()

,Жилая,Кухня,Общая,Цена,Цена за квадрат,Этаж,Этажность здания,Время,Способ передвижения,Широта,Долгота
0,24.0,10.0,48.0,6.55,136458.0,12,25,15.0,0,55.913162,37.713063
1,45.0,NaN,64.0,26.00,406250.0,4,7,13.0,1,55.750727,37.562272
2,23.4,8.9,40.0,18.00,450000.0,2,3,5.0,1,55.769216,37.617411
3,63.0,20.0,98.0,59.90,611224.0,33,54,11.0,0,55.723257,37.527660
4,48.0,NaN,67.3,37.90,563150.0,4,5,10.0,1,55.721731,37.572899


In [321]:
X = df.drop(columns=['Цена', 'Цена за квадрат'], axis=1)
y = df['Цена']

In [322]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.33, random_state=42)

In [323]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='median')

In [324]:
imputer.fit(X_train)
X_train = imputer.transform(X_train)
X_test = imputer.transform(X_test)

In [325]:
from sklearn import linear_model

In [326]:
lr = linear_model.LinearRegression()
lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [327]:
from sklearn import metrics

In [328]:
metrics.mean_absolute_error(y_train, lr.predict(X_train))

7.751037359329597

In [329]:
metrics.mean_absolute_error(y_test, lr.predict(X_test))

7.678177170991042

In [330]:
scale = preprocessing.StandardScaler()
scale.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [331]:
X_train_norm = scale.transform(X_train)
X_test_norm = scale.transform(X_test)

In [332]:
scale_target = preprocessing.StandardScaler()
scale_target.fit(y_train.values.reshape(-1, 1))

StandardScaler(copy=True, with_mean=True, with_std=True)

In [333]:
y_train_norm = scale_target.transform(y_train.values.reshape(-1, 1))
y_test_norm = scale_target.transform(y_test.values.reshape(-1, 1))

In [334]:
lr1 = linear_model.LinearRegression()
lr1.fit(X_train_norm, y_train_norm)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [335]:
metrics.mean_absolute_error(y_test_norm, lr1.predict(X_test_norm))

0.36673406990701507

In [336]:
lr_r = linear_model.Ridge(alpha=10.0)
lr_r.fit(X_train_norm, y_train)

Ridge(alpha=10.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [337]:
metrics.mean_absolute_error(y_train, lr_r.predict(X_train_norm))

7.749170908633046

In [338]:
metrics.mean_absolute_error(y_test, lr_r.predict(X_test_norm))

7.678327428776928

In [339]:
import xgboost

In [340]:
params = {
    "n_estimators": 1000,
    "max_depth": 10,
    "min_samples_split": 2,
    "learning_rate": 0.05,
}
xgb = xgboost.XGBRegressor(**params, silent=True)
xgb.fit(X_train_norm, y_train)

XGBRegressor(base_score=0.5, booster=None, colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
       importance_type='gain', interaction_constraints=None,
       learning_rate=0.05, max_delta_step=0, max_depth=10,
       min_child_weight=1, min_samples_split=2, missing=nan,
       monotone_constraints=None, n_estimators=1000, n_jobs=0,
       num_parallel_tree=1, objective='reg:squarederror', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, silent=True,
       subsample=1, tree_method=None, validate_parameters=False,
       verbosity=None)

In [341]:
metrics.mean_absolute_error(y_test, xgb.predict(X_test_norm))

3.08901970057062

In [342]:
poly = preprocessing.PolynomialFeatures(2, interaction_only=True)

In [343]:
xgb1 = xgboost.XGBRegressor(**params, silent=True)
xgb1.fit(poly.fit_transform(X_train_norm), y_train)

XGBRegressor(base_score=0.5, booster=None, colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
       importance_type='gain', interaction_constraints=None,
       learning_rate=0.05, max_delta_step=0, max_depth=10,
       min_child_weight=1, min_samples_split=2, missing=nan,
       monotone_constraints=None, n_estimators=1000, n_jobs=0,
       num_parallel_tree=1, objective='reg:squarederror', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, silent=True,
       subsample=1, tree_method=None, validate_parameters=False,
       verbosity=None)

In [344]:
metrics.mean_absolute_error(y_test, xgb1.predict(poly.fit_transform(X_test_norm)))

3.534173664612331